In [1]:
# Importing PlaidML. Make sure you follow this order
# import plaidml.keras
# plaidml.keras.install_backend()
# import os
# os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

# then you can write your codes
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Input
from keras.layers import Conv2D, MaxPooling2D, PReLU, AveragePooling2D, ZeroPadding2D
from keras import backend as K
from keras import optimizers

In [2]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import to_categorical

MNIST

In [3]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_imgs, train_labels), (test_imgs, test_labels) = mnist.load_data(path='mnist.npz')

# 灰度归一化
train_imgs = train_imgs / 255
test_imgs = test_imgs / 255

# 调整维度，按照 28*28 大小拆分图片数据

train_imgs = train_imgs.reshape(-1, 28, 28, 1)
test_imgs = test_imgs.reshape(-1, 28, 28, 1)

# 调整标签为onehot
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [4]:
import numpy as np

train_imgs = np.resize(train_imgs, (60000, 224, 224, 1))
test_imgs = np.resize(test_imgs, (10000, 224, 224, 1))

残差

In [5]:
def Conv_BN_Relu(filters, kernel_size, strides, input_shape):
    model = Sequential()
    model.add(Conv2D(filters, kernel_size, strides=strides, padding='same', input_shape=input_shape))
    # model.add(BatchNormalization())
    model.add(PReLU('ones'))
    model.add(MaxPooling2D(pool_size=3, strides=1))
    return model

def resiidual_a(pre_model, filters, input_shape, pre_index=0, index=3):
    model = Sequential()
    model.add(Conv_BN_Relu(filters, 3, 1, input_shape))
    if pre_model.output_shape == model.output_shape:
        print(pre_model.layers[pre_index], model.layers[index])
        model.add([pre_model.layers[pre_index], model.layers[index]])
    else:
        return model

    return model

AlexNet

In [6]:
model = Sequential()

model.add(Conv2D(96, kernel_size=11, strides=4, padding='same'
                 , input_shape=(224,224,1)
                 ))
model.add(PReLU('ones'))
model.add(MaxPooling2D(pool_size=3, strides=2))
model.add(resiidual_a(model, 256, model.output_shape[1:], 2, 3))
model.add(Conv2D(128, kernel_size=3, strides=1))
# model.add(BatchNormalization())
model.add(PReLU('ones'))
model.add(MaxPooling2D(pool_size=3, strides=1))
model.add(resiidual_a(model, 256, model.output_shape[1:], 6, 7))
model.add(Conv2D(384, kernel_size=3, strides=1))
# model.add(BatchNormalization())
model.add(PReLU('ones'))
model.add(MaxPooling2D(pool_size=3, strides=1))
model.add(Conv2D(256, kernel_size=3, strides=1))
# model.add(BatchNormalization())
model.add(PReLU('ones'))
model.add(MaxPooling2D(pool_size=3, strides=1))
model.add(resiidual_a(model, 384, model.output_shape[1:], 18, 21))

# 构建分类器
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

Metal device set to: Apple M2 Max


2023-05-05 11:30:39.875370: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-05 11:30:39.875500: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
# 训练模型
history = model.fit(train_imgs, train_labels, epochs=16, batch_size=128)
# 评估模型
evaluate_res = model.evaluate(x=test_imgs, y=test_labels, batch_size=128)

# 输出评估结果
print(evaluate_res)

# 输出模型的结构信息
model.summary()

Epoch 1/16


2023-05-05 11:30:52.257591: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-05 11:30:52.736084: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


283/469 [=================>............] - ETA: 1:16 - loss: 2.3029 - accuracy: 0.1088

KeyboardInterrupt: 

In [ ]:
# 借助 history 对象了解训练过程
history_dict = history.history
acc = history_dict['acc']
loss = history_dict['loss']

# 借助 Matplotlib 绘制图像
plt.plot(range(1, len(acc)+1), acc, 'b--')
plt.plot(range(1, len(loss)+1), loss, 'r-')

# 显示图例
plt.legend(['accuracy', 'loss'])

AlexNet

In [ ]:
train_imgs = train_imgs.copy().resize(train_imgs.shape[0], 224, 224, 1)
test_imgs = test_imgs.copy().resize(test_imgs.shape[0], 224, 224, 1)

Cifar 10

In [3]:
from keras.datasets import cifar10
from keras.utils import to_categorical

In [10]:
(train_imgs, train_labels), (test_imgs, test_labels) = cifar10.load_data()

train_imgs = train_imgs / 255
test_imgs = test_imgs / 255

train_imgs = train_imgs.reshape(-1, 32, 32, 3)
test_imgs = test_imgs.reshape(-1, 32, 32, 3)

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [22]:
model = Sequential()

model.add(Conv2D(96,(3,3), input_shape=(32,32,3)))
model.add(PReLU('ones'))
# model.add(BatchNormalization(axis=-1,
#     momentum=0.99,
#     epsilon=0.001,))
model.add(MaxPooling2D(pool_size=(2, 2),strides=1))

model.add(Conv2D(512,(3,3)))
model.add(PReLU('ones'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256,(3,3)))
model.add(PReLU('ones'))
# model.add(BatchNormalization(axis=-1,
#     momentum=0.99,
#     epsilon=0.001,))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512,(3,3)))
model.add(PReLU('ones'))
# model.add(BatchNormalization(0.5))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(500, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 30, 30, 96)        2688      
_________________________________________________________________
p_re_lu_26 (PReLU)           (None, 30, 30, 96)        86400     
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 29, 29, 96)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 27, 27, 512)       442880    
_________________________________________________________________
p_re_lu_27 (PReLU)           (None, 27, 27, 512)       373248    
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 13, 13, 512)       0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 11, 11, 256)       1179904   
__________

In [ ]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
history = model.fit(train_imgs, train_labels, epochs=32, batch_size=64, shuffle=True)
# 评估模型
evaluate_res = model.evaluate(x=test_imgs, y=test_labels, batch_size=64)

# 输出评估结果
print(evaluate_res)

Epoch 1/32
50000/50000 [==============================] - 166s 3ms/step - loss: 0.6499 - acc: 0.7728
Epoch 2/32
50000/50000 [==============================] - 157s 3ms/step - loss: 0.6178 - acc: 0.7819
Epoch 3/32
50000/50000 [==============================] - 189s 4ms/step - loss: 0.5879 - acc: 0.7908
Epoch 4/32
50000/50000 [==============================] - 230s 5ms/step - loss: 0.5649 - acc: 0.8000
Epoch 5/32
50000/50000 [==============================] - 267s 5ms/step - loss: 0.5345 - acc: 0.8118
Epoch 6/32
50000/50000 [==============================] - 316s 6ms/step - loss: 0.5140 - acc: 0.8198
Epoch 7/32
50000/50000 [==============================] - 326s 7ms/step - loss: 0.4795 - acc: 0.8317
Epoch 8/32
50000/50000 [==============================] - 322s 6ms/step - loss: 0.4578 - acc: 0.8400
Epoch 9/32
50000/50000 [==============================] - 304s 6ms/step - loss: 0.4279 - acc: 0.8487
Epoch 10/32
50000/50000 [==============================] - 284s 6ms/step - loss: 0.4056 - a

In [ ]:
# 借助 history 对象了解训练过程
history_dict = history.history
acc = history_dict['acc']
loss = history_dict['loss']

# 借助 Matplotlib 绘制图像
plt.plot(range(1, len(acc)+1), acc, 'b--')
plt.plot(range(1, len(loss)+1), loss, 'r-')

# 显示图例
plt.legend(['accuracy', 'loss'])

Fashion_MNIST

In [ ]:
from keras.datasets import fashion_mnist
from keras.utils import to_categorical

(train_imgs, train_labels), (test_imgs, test_labels) = fashion_mnist.load_data()

# 灰度归一化
train_imgs = train_imgs / 255
test_imgs = test_imgs / 255

# 调整维度，按照 28*28 大小拆分图片数据

train_imgs = train_imgs.reshape(-1, 28, 28, 1)
test_imgs = test_imgs.reshape(-1, 28, 28, 1)

# 调整标签为onehot
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout
from keras.layers.merge import concatenate

In [ ]:
def Inception_block(input_layer, f1, f2_conv1, f2_conv3, f3_conv1, f3_conv5, f4, f5):
  # Input:
  # - f1: number of filters of the 1x1 convolutional layer in the first path
  # - f2_conv1, f2_conv3 are number of filters corresponding to the 1x1 and 3x3 convolutional layers in the second path
  # - f3_conv1, f3_conv5 are the number of filters corresponding to the 1x1 and 5x5  convolutional layer in the third path
  # - f4: number of filters of the 1x1 convolutional layer in the fourth path

  # 1st path:
  path1 = Conv2D(filters=f1, kernel_size = (1,1), padding = 'same')(input_layer)
  path1 = PReLU('one')(path1)

  # 2nd path
  path2 = Conv2D(filters = f2_conv1, kernel_size = (1,1), padding = 'same')(input_layer)
  path2 = PReLU('one')(path2)
  path2 = Conv2D(filters = f2_conv3, kernel_size = (3,3), padding = 'same')(path2)
  path2 = PReLU('one')(path2)

  # 3rd path
  path3 = Conv2D(filters = f3_conv1, kernel_size = (1,1), padding = 'same')(input_layer)
  path3 = PReLU('one')(path3)
  path3 = Conv2D(filters = f3_conv5, kernel_size = (5,5), padding = 'same')(path3)
  path3 = PReLU('one')(path3)

  # 4th path
  path4 = MaxPooling2D((3,3), strides= (1,1), padding = 'same')(input_layer)
  path4 = Conv2D(filters = f4, kernel_size = (1,1), padding = 'same')(path4)
  path4 = PReLU('one')(path4)

  # 5th path
  path5 = AveragePooling2D(3, strides=1, padding = 'same')(input_layer)
  path5 = Conv2D(filters = f5, kernel_size=1, padding = 'same')(path5)
  path5 = PReLU('ones')(path5)

  output_layer = concatenate([path1, path2, path3, path4, path5], axis = -1)

  return output_layer

In [ ]:
def GoogLeNet():

  input_layer = Input(shape = (28, 28, 1))
  X = Conv2D(filters = 64, kernel_size = (3,3), strides = 1, padding = 'valid')(input_layer)
  X = PReLU('ones')(X)
  X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)
  X = Conv2D(filters = 192, kernel_size = (3,3), padding = 'same')(X)
  X = PReLU('ones')(X)
  X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

  X = Inception_block(X, f1 = 64, f2_conv1 = 96, f2_conv3 = 128, f3_conv1 = 16, f3_conv5 = 32, f4 = 32, f5=32)
  X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 192, f3_conv1 = 32, f3_conv5 = 96, f4 = 64, f5=32)
  # X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)
  # X = Inception_block(X, f1 = 192, f2_conv1 = 96, f2_conv3 = 208, f3_conv1 = 16, f3_conv5 = 48, f4 = 64)

  # X = Inception_block(X, f1 = 160, f2_conv1 = 112, f2_conv3 = 224, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)
  # X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 256, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)
  # X = Inception_block(X, f1 = 112, f2_conv1 = 144, f2_conv3 = 288, f3_conv1 = 32, f3_conv5 = 64, f4 = 64)

  X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, f3_conv5 = 128, f4 = 128, f5=64)
  X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)
  X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, f3_conv5 = 128, f4 = 128, f5=64)
  X = Inception_block(X, f1 = 384, f2_conv1 = 192, f2_conv3 = 384, f3_conv1 = 48, f3_conv5 = 128, f4 = 128, f5=64)

  # Global Average pooling layer
  X = GlobalAveragePooling2D(name = 'GAPL')(X)
  X = Dropout(0.4)(X)
  X = Dense(10, activation = 'softmax')(X)

  # model
  model = Model(input_layer, [X], name = 'GoogLeNet')

  return model

In [ ]:
model = GoogLeNet()
model.summary()

In [ ]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
history = model.fit(train_imgs, train_labels, epochs=24, batch_size=64, shuffle=True)
# 评估模型
evaluate_res = model.evaluate(x=test_imgs, y=test_labels, batch_size=64)

# 输出评估结果
print(evaluate_res)

In [ ]:
history_dict = history.history
acc = history_dict['acc']
loss = history_dict['loss']

# 借助 Matplotlib 绘制图像
plt.plot(range(1, len(acc)+1), acc, 'b--')
plt.plot(range(1, len(loss)+1), loss, 'r-')

# 显示图例
plt.legend(['accuracy', 'loss'])